# Install Libraries

In [ ]:
!pip install -q U transformers langchain langchain-huggingface huggingface_hub accelerate bitsandbytes langchain_community

# Login HuggingFace and Add API Key

In [1]:
from huggingface_hub import notebook_login
notebook_login()

# Import Libraries

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForSequenceClassification, BitsAndBytesConfig,pipeline,AutoModelForCausalLM
import torch
from langchain import LLMChain,PromptTemplate
from langchain_huggingface import HuggingFacePipeline
import warnings
warnings.filterwarnings("ignore")

2024-06-11 15:50:49.432864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 15:50:49.433002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 15:50:49.565425: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Model For Summarization

In [3]:
summarization_model_id='facebook/bart-large-cnn'
summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model_id)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_id)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

# Model For Sentiment Analysis

In [4]:
sentiment_model_id='mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_id)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_id)

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

# Model For Text Generation


In [5]:
bnb_config= BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_compute_dtype=torch.bfloat16,
                              bnb_4bit_quant_type='nf4')

text_gen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B")
text_gen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B",quantization_config=bnb_config,device_map=0)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

# Use HuggingFacePipeline to create pipeline

In [6]:
summarization_pipe= pipeline(task='summarization',model=summarization_model,tokenizer=summarization_tokenizer,max_length=100,temperature=0.1)
summarization_pipeline= HuggingFacePipeline(pipeline=summarization_pipe)

# Use huggingface transformers pipeline directly for sentiment

In [7]:
sentiment_pipeline=  pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

# Use HuggingFacePipeline to create Text Gen pipeline


In [8]:
text_gen_pipe= pipeline(task= 'text-generation', model=text_gen_model,tokenizer=text_gen_tokenizer,max_new_tokens=400,temperature=0.1)
text_gen_pipeline= HuggingFacePipeline(pipeline=text_gen_pipe)

# Prompt Template for Summarization

In [9]:
summarization_template= "You are a text summarizer.just summarize the {review}"
summarization_prompt= PromptTemplate(template=summarization_template,input_variables=['review'])


# Prompt Template for Text Generation

In [10]:
text_gen_template= "You are a text generator.so give list os 5 short reason of negative or positive and give area of improvements for reviews"
text_gen_prompt= PromptTemplate(template=text_gen_template,input_variables=['reviews'])

# Create Chain 

In [11]:
summarization_chain= summarization_prompt | summarization_pipeline

In [12]:
text_gen_chain= text_gen_prompt | text_gen_pipeline

# Sample review

In [13]:
reviews = ['''I had high hopes for the PQR laptop, but unfortunately, it turned out to be a complete disaster. From the moment I opened the box, I was disappointed. The design looked outdated and cheap, nothing like the sleek and modern laptop I was expecting. Upon booting it up, I was greeted with sluggish performance and constant freezing. Simple tasks like browsing the web or typing up a document took ages to complete, leaving me frustrated and unproductive.
The build quality was also subpar. The plastic casing felt flimsy and easily scratched, and the keyboard had a cheap, plasticky feel to it. The trackpad was unresponsive and frequently malfunctioned, making navigation a nightmare. Even worse, the battery life was laughable. Despite claims of long-lasting power, the battery drained within a few hours of light usage, forcing me to constantly hunt for a power outlet.
But perhaps the biggest disappointment was the display. The screen was dim and washed out, with poor color accuracy and terrible viewing angles. Watching movies or editing photos was a dismal experience, as everything looked dull and lifeless. And don't even get me started on the sound quality. The speakers were tinny and distorted, making music and videos sound like they were being played through a cheap transistor radio.
To make matters worse, the customer service was abysmal. When I reached out to the company for help with my issues, I was met with indifference and incompetence. Emails went unanswered, and phone calls were met with endless hold times and unhelpful representatives. It was clear that they had no interest in standing behind their product or ensuring customer satisfaction''']

# Model Inferencing

In [14]:
 for review in reviews:
    summary= summarization_chain.invoke(review)
    sentiment= sentiment_pipeline(summary)
    text_gen= text_gen_chain.invoke({'review':reviews})
    
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('Review:',review)
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('Summary:',summary)
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('Sentiment:',sentiment[0]['label'])
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('text_gen:',text_gen)
    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------------------------------------------------')

-----------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------------------------
Review: I had high hopes for the PQR laptop, but unfortunately, it turned out to be a complete disaster. From the moment I opened the box, I was disappointed. The design looked outdated and cheap, nothing like the sleek and modern laptop I was expecting. Upon booting it up, I was greeted with sluggish performance and constant freezing. Simple tasks like browsing the web or typing up a document took ages to complete, leaving me frustrated and unproductive.
The build quality was also subpar. The plastic casing felt flimsy and easily scratched, and the keyboard had a cheap, plasticky feel to it. The trackpad was unresponsive and frequently malfunctioned, making navigation a nightmare. Even worse, the battery life was laughable. Despite claim